In [5]:
from transformers import HubertModel, HubertConfig
from DatasetLoader import loadWAV
from pathlib import Path
import glob
import os
import numpy as np
import torch
import random
from torch import nn
import time
import sys
from speechbrain.lobes.models import ECAPA_TDNN
from DatasetLoader import train_dataset_loader, test_dataset_loader, train_dataset_sampler, worker_init_fn
from torch.utils.data import DataLoader
from loss.aamsoftmax import LossFunction as AAM
import itertools
import torch.nn.functional as F
from tuneThreshold import *
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import autocast, GradScaler

/home/iref/Repos/voxceleb_trainer/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
wavs_list = glob.glob("./data/voxceleb2/*/*/*.wav")
test = [np.random.choice(wavs_list) for t in range(4)]
waveforms = [loadWAV(t, 200, False) for t in test]
print(waveforms[0].shape)
tensor = torch.tensor(waveforms, dtype=torch.float, device=device).squeeze(1)
configuration = HubertConfig()
model = HubertModel(configuration)
model.to(device)
print(tensor.shape)
out = model(tensor, output_hidden_states=True)
hs = out.hidden_states
print(len(hs))
for st in hs:
    print(st.shape)
print()
print(out.last_hidden_state.shape)

(1, 32240)


/tmp/ipykernel_3028/2075572349.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  tensor = torch.tensor(waveforms, dtype=torch.float, device=device).squeeze(1)


torch.Size([4, 32240])
13
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])
torch.Size([4, 100, 768])

torch.Size([4, 100, 768])


In [3]:
temp = torch.stack(hs).squeeze(1).sum(0)
temp.shape

NameError: name 'hs' is not defined

In [10]:
class HuCapa(nn.Module):
    def __init__(self, device) -> None:
        super().__init__()
        self.hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960")
        self.hubert.config.output_hidden_states = True
        self.hubert.to(device)
        self.hubert.eval()
        self.hubert.requires_grad_(False)
        
        hs_num = len(self.hubert.encoder.layers) + 1
        #self.hs_weights = nn.Parameter(torch.ones(hs_num, dtype=torch.float, device=device, requires_grad=True))
        self.hs_weights = nn.Linear(13, 1, bias=False, device=device)

        self.ecapa = ECAPA_TDNN.ECAPA_TDNN(768)
        self.ecapa.to(device)

    def forward(self, x):
        backbone_embs = self.hubert(x).hidden_states
        #print(backbone_embs[0].shape)
        backbone_embs = torch.stack(backbone_embs).squeeze(1)
        backbone_embs = torch.transpose(backbone_embs, 0, 1)
        backbone_embs = torch.transpose(backbone_embs, 3, 1)
        #print(backbone_embs.shape)
        linear_comb_hs = self.hs_weights(backbone_embs)
        #linear_comb_hs = backbone_embs * self.hs_weights[:, None,  None]
        #print(linear_comb_hs.shape)
        #linear_comb_hs = linear_comb_hs.sum(1)
        #print(linear_comb_hs.shape)
        linear_comb_hs = linear_comb_hs.squeeze(3)
        linear_comb_hs = linear_comb_hs.transpose(2, 1)
        #print(linear_comb_hs.shape)
        ecapa_embs = self.ecapa(linear_comb_hs)
        return ecapa_embs
        
        

In [20]:
test_model = HuCapa(device=device)
#test_model(tensor).shape

In [ ]:
test_model

HuCapa(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (2): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (3): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (4): HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation):

In [6]:
test_model.hubert.encoder.layers[0]

HubertEncoderLayer(
  (attention): HubertAttention(
    (k_proj): Linear(in_features=768, out_features=768, bias=True)
    (v_proj): Linear(in_features=768, out_features=768, bias=True)
    (q_proj): Linear(in_features=768, out_features=768, bias=True)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (feed_forward): HubertFeedForward(
    (intermediate_dropout): Dropout(p=0.1, inplace=False)
    (intermediate_dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
    (output_dense): Linear(in_features=3072, out_features=768, bias=True)
    (output_dropout): Dropout(p=0.1, inplace=False)
  )
  (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [1]:
def evaluateFromList(test_list, test_path, model, nDataLoaderThread, print_interval=100, num_eval=10, eval_frames=300):

        model.ecapa.eval()
        model.hs_weights.eval()

        lines = []
        files = []
        feats = {}
        tstart = time.time()
        cos =  nn.CosineSimilarity(dim=0) 

        ## Read all lines
        with open(test_list) as f:
            lines = f.readlines()

        ## Get a list of unique file names
        files = list(itertools.chain(*[x.strip().split()[-2:] for x in lines]))
        setfiles = list(set(files))
        setfiles.sort()

        ## Define test data loader
        test_dataset = test_dataset_loader(setfiles, test_path, num_eval=num_eval, eval_frames=eval_frames)

        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=nDataLoaderThread, drop_last=False, sampler=None)

        ## Extract features for every image
        for idx, data in enumerate(test_loader):
            inp1 = data[0][0].cuda()
            with torch.no_grad():
                ref_feat = model(inp1).detach().cpu()
            feats[data[1][0]] = ref_feat
            telapsed = time.time() - tstart
            

            if idx % print_interval == 0:
                sys.stdout.write(
                    "\rReading {:d} of {:d}: {:.2f} Hz, embedding size {:d}\n".format(idx, test_loader.__len__(), idx / telapsed, ref_feat.size()[2])
                )
                #print(ref_feat.size())

        print(next(iter(feats.items())))
        all_scores = []
        all_labels = []
        all_trials = []

        tstart = time.time()
        print("")

        ## Read files and compute all scores
        for idx, line in enumerate(lines):

            data = line.split()

            ## Append random label if missing
            if len(data) == 2:
                data = [random.randint(0, 1)] + data

            ref_feat = feats[data[1]].cuda()
            com_feat = feats[data[2]].cuda()



   
            ref_feat = F.normalize(ref_feat, p=2, dim=0)
            com_feat = F.normalize(com_feat, p=2, dim=0)



            dist = torch.cdist(ref_feat.reshape(num_eval, -1), com_feat.reshape(num_eval, -1)).detach().cpu().numpy()
            #dist = cos(ref_feat.reshape(num_eval, -1), com_feat.reshape(num_eval, -1)).detach().cpu().numpy()
            #print(dist)

            score = -1 * np.mean(dist)
            #score = np.mean(dist)

            all_scores.append(score)
            all_labels.append(int(data[0]))
            all_trials.append(data[1] + " " + data[2])

            if idx % print_interval == 0:
                telapsed = time.time() - tstart
                #print(ref_feat.shape)
                #print(com_feat.shape)
                #print(ref_feat.reshape(num_eval, -1).shape)
                #print(com_feat.reshape(num_eval, -1).shape)
                print(score)

                sys.stdout.write("\rComputing {:d} of {:d}: {:.2f} Hz\n".format(idx, len(lines), idx / telapsed))
                sys.stdout.flush()
                print("#"*64)

        return (all_scores, all_labels, all_trials, feats)

In [12]:
class HuCapaTrainer(nn.Module):
    def __init__(
        self, 
        max_frames: int=500, 
        device:torch.device=torch.device("cuda"), 
        batch_size:int = 32, 
        num_wokers:int=16, 
        seed:int=10,
        lr:float=1e-3,
        test_step:int=1,
        scheduler_type = "step",
        unfreeze_hubert = False,
        lr_decay:float=0.97
    ) -> None:
        super().__init__()
        self.device = device
        self.model = HuCapa(self.device)
        train_dataset = train_dataset_loader(
            train_list="data/train_list.txt",
            augment=True,
            musan_path="./data/musan_split",
            rir_path="./data/RIRS_NOISES/simulated_rirs",
            max_frames=max_frames,
            train_path="data/voxceleb2"
            )
        train_sampler = train_dataset_sampler(
            train_dataset,
            nPerSpeaker=1, 
            max_seg_per_spk=500, 
            batch_size=batch_size,
            distributed=False,
            seed=seed
            )
        self.dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            num_workers=num_wokers,
            sampler=train_sampler,
            pin_memory=False,
            worker_init_fn=worker_init_fn,
            drop_last=True
            )
        self.test_file = "./data/test_list.txt"
        self.test_path = "./data/voxceleb1/"
        self.loss = AAM(nOut=192, nClasses=5994, margin=0.2, scale=30)
        self.optim = torch.optim.Adam([p for p in itertools.chain(self.model.parameters(), self.loss.parameters()) if p.requires_grad], lr=lr, weight_decay=2e-5)
        if scheduler_type == "step":
            self.scheduler = torch.optim.lr_scheduler.StepLR(self.optim, step_size=test_step, gamma=lr_decay)
        elif scheduler_type == "cycle":
            self.scheduler = torch.optim.lr_scheduler.CyclicLR(self.optim, base_lr=1e-8, max_lr=1e-6, step_size_up=2729, mode="triangular2", cycle_momentum=False)
        self.scaler = GradScaler()

        if unfreeze_hubert:
            for layer in [0, 1, 3, -2]:
                self.model.hubert.encoder.layers[layer].requires_grad_(True)

        print(time.strftime("%m-%d %H:%M:%S") + " Overall parameters: = %.2f"%(sum(param.numel() for param in self.model.parameters())))
        print(f"Learnable parameters: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}")

    def load_params(self, path):
        checkpoint = torch.load(path)
        self.model.ecapa.load_state_dict(checkpoint["ecapa"])
        self.model.hs_weights.load_state_dict(checkpoint["hs_weights"])
        self.optim.load_state_dict(checkpoint["opt"])
        self.loss.load_state_dict(checkpoint["loss"])
        if "scaler" in checkpoint:
            self.scaler.load_state_dict(checkpoint["scaler"])
        return checkpoint["epoch"]

    def train(self, epoch):
        self.model.ecapa.train()
        index, top1, loss = 0, 0, 0
        lr = self.optim.param_groups[0]["lr"]
        for num, (data, labels) in enumerate(self.dataloader, start=1):
            self.zero_grad()
            data = data.to(self.device)
            data = data.squeeze(1)
            labels = torch.tensor(labels, dtype=torch.long, device=self.device)
            print(labels.shape)
            with autocast():
                out = self.model(data)
                print(out.shape)
                out = out.squeeze(1)
                nloss, prec = self.loss(out, labels)
            self.scaler.scale(nloss).backward()
            self.scaler.step(self.optim)
            self.scaler.update()
            #self.optim.step()
            index += len(labels)
            top1 += prec
            loss += nloss.detach().cpu().numpy()
            sys.stderr.write(time.strftime("%m-%d %H:%M:%S") + \
            " [%2d] Lr: %5f, Training: %.2f%%, "    %(epoch, lr, 100 * (num / self.dataloader.__len__())) + \
            " Loss: %.5f, ACC: %2.2f%% \r"        %(loss/(num), top1/index*len(labels)))
            sys.stderr.flush()
            self.scheduler.step()
        sys.stdout.write("\n")
        return loss / num, lr, top1 / index*len(labels)


    def eval(self, epoch):
        self.model.ecapa.eval()
        self.model.hs_weights.eval()
        self.loss.eval()
        sc, lab, _ = evaluateFromList(self.test_file, self.test_path, self.model, self.dataloader.num_workers)
        result = tuneThresholdfromScore(sc, lab, [1, 0.1])
        fnrs, fprs, thresholds = ComputeErrorRates(sc, lab)
        mindcf, _ = ComputeMinDcf(fnrs, fprs, thresholds, p_target=0.05, c_miss=1, c_fa=1)
        print('\n',time.strftime("%Y-%m-%d %H:%M:%S"), "Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}".format(epoch, result[1], mindcf))
        return result[1], mindcf

In [13]:
def train_wrapper(start_epoch: int=0, epochs_num: int=2, test_every: int=5, cpt_path: str = ""):
    trainer = HuCapaTrainer(batch_size=128)
    if cpt_path:
        start_epoch = trainer.load_params(cpt_path)
    start_epoch +=1
    save_path = Path(f"exps/HuCapa/diff/{start_epoch}-{start_epoch + epochs_num - 1}")
    os.makedirs(save_path, exist_ok=True)
    writer = SummaryWriter(log_dir=save_path / "tb_logs")
    scorefile = open(save_path / "scores.txt", "a+")
    for epoch in range(start_epoch, start_epoch + epochs_num):
        loss, lr, acc = trainer.train(epoch)
        writer.add_scalar("Train loss", loss, epoch)
        writer.add_scalar("Train acc/eer", acc, epoch)
        writer.add_scalar("LR", lr, epoch)
        print(f"hidden states weights: {trainer.model.hs_weights.weight}")
        scorefile.write("Epoch {:d}, TEER/TAcc {:2.2f}, TLOSS {:f}, LR {:f} \n".format(epoch, acc.item(), loss, lr))

        if epoch % test_every == 0:
            val_eer, dcf = trainer.eval(epoch)
            writer.add_scalar("Val EER", val_eer, epoch)
            writer.add_scalar("Val minDCF", dcf, epoch)
            scorefile.write("Epoch {:d}, VEER {:2.4f}, MinDCF {:2.5f}\n".format(epoch, val_eer, dcf))
            torch.save({
                "ecapa": trainer.model.ecapa.state_dict(),
                "hs_weights": trainer.model.hs_weights.state_dict(),
                "opt": trainer.optim.state_dict(),
                "epoch": epoch 
            }, save_path / f"{start_epoch}-{start_epoch + epochs_num - 1}_cp.tar")
            scorefile.flush()
    scorefile.close()



In [14]:
train_wrapper(epochs_num=2, test_every=1, cpt_path="")

Initialised AAMSoftmax margin 0.200 scale 30.000
02-01 20:40:21 Overall parameters: = 102327053.00
Learnable parameters: 7955341


/tmp/ipykernel_16441/1346584451.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long, device=self.device)


torch.Size([128])
torch.Size([128, 1, 192])


/home/iref/Repos/voxceleb_trainer/env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


torch.Size([128])


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.92 GiB (GPU 0; 23.70 GiB total capacity; 6.75 GiB already allocated; 3.13 GiB free; 13.66 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [30]:
trainer = HuCapaTrainer(batch_size=128)
trainer.load_params("/home/iref/Repos/voxceleb_trainer/exps/HuCapa/cyclic_sched/10-12/10-12_cp.tar")
scores, labels, trials, feats = evaluateFromList(trainer.test_file, trainer.test_path, trainer.model, 16)


Initialised AAMSoftmax margin 0.200 scale 30.000
01-30 17:07:17 Overall parameters: = 102327053.00
Learnable parameters: 7955341
Reading 0 of 4715: 0.00 Hz, embedding size 192
Reading 100 of 4715: 25.02 Hz, embedding size 192
Reading 200 of 4715: 25.93 Hz, embedding size 192
Reading 300 of 4715: 26.21 Hz, embedding size 192
Reading 400 of 4715: 26.40 Hz, embedding size 192
Reading 500 of 4715: 26.52 Hz, embedding size 192
Reading 600 of 4715: 26.61 Hz, embedding size 192
Reading 700 of 4715: 26.66 Hz, embedding size 192
Reading 800 of 4715: 26.69 Hz, embedding size 192
Reading 900 of 4715: 26.74 Hz, embedding size 192
Reading 1000 of 4715: 26.75 Hz, embedding size 192
Reading 1100 of 4715: 26.78 Hz, embedding size 192
Reading 1200 of 4715: 26.80 Hz, embedding size 192
Reading 1300 of 4715: 26.81 Hz, embedding size 192
Reading 1400 of 4715: 26.83 Hz, embedding size 192
Reading 1500 of 4715: 26.85 Hz, embedding size 192
Reading 1600 of 4715: 26.86 Hz, embedding size 192
Reading 1700 of 4

In [77]:
trials

['id10270/x6uYqmx31kE/00001.wav id10270/8jEAjG6SegY/00008.wav',
 'id10270/x6uYqmx31kE/00001.wav id10300/ize_eiCFEg0/00003.wav',
 'id10270/x6uYqmx31kE/00001.wav id10270/GWXujl-xAVM/00017.wav',
 'id10270/x6uYqmx31kE/00001.wav id10273/0OCW1HUxZyg/00001.wav',
 'id10270/x6uYqmx31kE/00001.wav id10270/8jEAjG6SegY/00022.wav',
 'id10270/x6uYqmx31kE/00001.wav id10284/Uzxv7Axh3Z8/00001.wav',
 'id10270/x6uYqmx31kE/00001.wav id10270/GWXujl-xAVM/00033.wav',
 'id10270/x6uYqmx31kE/00001.wav id10284/7yx9A0yzLYk/00029.wav',
 'id10270/x6uYqmx31kE/00002.wav id10270/5r0dWxy17C8/00026.wav',
 'id10270/x6uYqmx31kE/00002.wav id10285/m-uILToQ9ss/00009.wav',
 'id10270/x6uYqmx31kE/00002.wav id10270/GWXujl-xAVM/00035.wav',
 'id10270/x6uYqmx31kE/00002.wav id10306/uzt36PBzT2w/00001.wav',
 'id10270/x6uYqmx31kE/00002.wav id10270/GWXujl-xAVM/00038.wav',
 'id10270/x6uYqmx31kE/00002.wav id10307/kp_GCjLq4qA/00004.wav',
 'id10270/x6uYqmx31kE/00002.wav id10270/GWXujl-xAVM/00033.wav',
 'id10270/x6uYqmx31kE/00002.wav id10275/

In [31]:
min(scores), max(scores)

(-6.8398756980896, -3.234910488128662)

In [32]:
for el in zip(scores[30:40], labels[30:40]):
    print(el)

(-4.743126392364502, 1)
(-5.972294330596924, 0)
(-5.445388317108154, 1)
(-6.108090400695801, 0)
(-4.902193069458008, 1)
(-6.2031121253967285, 0)
(-4.818323612213135, 1)
(-5.260000705718994, 0)
(-5.194511890411377, 1)
(-6.3476738929748535, 0)


In [33]:
result = tuneThresholdfromScore(scores, labels, [1, 0.1])
fnrs, fprs, thresholds = ComputeErrorRates(scores, labels)
mindcf, _ = ComputeMinDcf(fnrs, fprs, thresholds, p_target=0.05, c_miss=1, c_fa=1)

In [34]:
result[1], mindcf

(2.290562036055143, 0.16527041357370006)

In [22]:
list(trainer.model.hs_weights.parameters())

[Parameter containing:
 tensor([[2.1916, 1.0707, 0.9042, 0.9030, 0.9885, 0.8775, 0.6379, 0.5067, 0.3276,
          0.1593, 0.3759, 1.0025, 0.0062]], device='cuda:0', requires_grad=True)]

In [24]:
list(test_model.hs_weights.parameters())

[Parameter containing:
 tensor([[-0.1515,  0.1714, -0.0056,  0.2554,  0.1831, -0.0230, -0.1913, -0.2452,
           0.1402,  0.1144,  0.0384,  0.2772, -0.2237]], device='cuda:0',
        requires_grad=True)]